In [1]:
import pandas as pd
import numpy as np

pr_gen_fuel = pd.read_pickle('../data/raw_eia923__puerto_rico_generation_fuel')
pr_plant_frame = pd.read_pickle('../data/raw_eia923__puerto_rico_plant_frame')

In [ ]:
# Handle EIA null values
pr_gen_fuel = pr_gen_fuel.replace(to_replace = ".", value = pd.NA)
# Convert data types (mmbtu/units to numeric)
pr_gen_fuel = pr_gen_fuel.convert_dtypes()

# Pivot fuel_consumed_for_electricity columns
## Only keep the index and relevant fuel_consumed columns
column_subset_list = ['plant_id_eia', 'plant_name_eia', 'report_year', 'prime_mover_code', 'energy_source_code'] + [col for col in pr_gen_fuel.columns if "fuel_consumed_for_electricity" in col]
pr_gen_fuel_elec = pr_gen_fuel.loc[:, column_subset_list]

## Stack the fuel_consumed columns
pr_gen_fuel_elec_melt = pr_gen_fuel_elec.melt(id_vars = ['report_year', 'plant_id_eia', 'plant_name_eia', 'prime_mover_code', 'energy_source_code'])
## Split the month from the variable
pr_gen_fuel_elec_melt[['fuel_variable', 'month']] = pr_gen_fuel_elec_melt['variable'].str.rsplit("_", n=1, expand=True)
## Create date from month and year
pr_gen_fuel_elec_melt['date'] = pd.to_datetime(pr_gen_fuel_elec_melt['month'] + pr_gen_fuel_elec_melt['report_year'].astype(str), format='%B%Y')
## Drop some stuff
pr_gen_fuel_elec_clean = pr_gen_fuel_elec_melt.drop(columns = ['report_year', 'month', 'variable'])

## Drop a bad plant
pr_gen_fuel_elec_cleaner = pr_gen_fuel_elec_clean.loc[(pr_gen_fuel_elec_clean.plant_id_eia != 62410) & (pr_gen_fuel_elec_clean.date.dt.year != 2020) & (pr_gen_fuel_elec_clean.value.notnull())]

## Split mmbtu and units into two columns
pr_gen_fuel_elec_cleaner.pivot(index=['plant_id_eia', 'plant_name_eia', 'prime_mover_code', 'energy_source_code','date'], columns = ['fuel_variable'], values='value').reset_index()

In [135]:
pr_gen_fuel_elec_cleaner

,plant_id_eia,plant_name_eia,prime_mover_code,energy_source_code,value,fuel_variable,date
0,61014,Pattern Santa Isabel LLC,WT,WND,101260.0,fuel_consumed_for_electricity_mmbtu,2017-04-01
1,61034,EcoElectrica,CA,NG,0.0,fuel_consumed_for_electricity_mmbtu,2017-04-01
2,61034,EcoElectrica,CT,NG,1976130.0,fuel_consumed_for_electricity_mmbtu,2017-04-01
3,61036,AES ILUMINA,PV,SUN,31886.0,fuel_consumed_for_electricity_mmbtu,2017-04-01
4,61082,AES Puerto Rico,ST,BIT,3258736.0,fuel_consumed_for_electricity_mmbtu,2017-04-01
...,...,...,...,...,...,...,...
10762,61149,Palo Seco Plant,GT,DFO,57260.0,fuel_consumed_for_electricity_units,2024-09-01
10763,61149,Palo Seco Plant,ST,RFO,165270.0,fuel_consumed_for_electricity_units,2024-09-01
10764,61150,Cambalache Plant,GT,DFO,109269.0,fuel_consumed_for_electricity_units,2024-09-01
10765,61151,Mayaguez Plant,GT,DFO,90693.0,fuel_consumed_for_electricity_units,2024-09-01
